In [60]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
!pip install tf-models-official
!pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 85579, done.
remote: Counting objects: 100% (1178/1178), done.
remote: Compressing objects: 100% (461/461), done.
remote: Total 85579 (delta 728), reused 1129 (delta 711), pack-reused 84401
Receiving objects: 100% (85579/85579), 598.64 MiB | 17.56 MiB/s, done.
Resolving deltas: 100% (61221/61221), done.


In [63]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research/models/research/models/research/models/research


In [64]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [65]:
!pip install lvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
import cv2
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.data import sampler
import os
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import PIL
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F 
import pandas as pd
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import tensorflow as tf
from object_detection import model_lib_v2
from object_detection.configs import tf2

In [68]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [69]:
import glob

file_paths = glob.glob('/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/*.jpg')
sorted_file_paths = sorted(file_paths)

for file_path in sorted_file_paths:
    # Process the files in sorted order
    print(file_path)

/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset102_png.rf.F8h03ElvOYmfCAosCW17.jpg
/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset102_png.rf.c990cebbb0907d6c38170fe7d1156886.jpg
/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset106_png.rf.VXIYHcr4exuziQCMkota.jpg
/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset106_png.rf.df293bd4ab73c822ff39d0a5622b4b68.jpg
/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset113_png.rf.M3Sr9y8TD2OIEBqPYafv.jpg
/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset113_png.rf.cd73b9340f7d440ebe52b7c6caa4a2a0.jpg
/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset116_png.rf.2b9266fb3d548ea8c731c4644ddf5afb.jpg
/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset116_png.rf.Rjc9Ur1arCER3krnJqJR.jpg
/content/gdrive/My Drive/SeaLifeClassificationDetection/valid/FishDataset122_png

In [70]:
weights = None
class CustomImageDataset():
    def __init__(self, root, mode = 'test', transform=None):
        self.ROOT = root
        self.d = [0,0,0]
        self.mode = mode
        if self.mode == 'test':
            self.d = [0,0]
        self.data, self.labels = self.create_data() 
       
        
        self.train_size = 1500
        
        self.mean, self.std = self.compute_stats(np.array(self.data))
        
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize(self.mean,self.std)
        ])
        
        if mode == 'train':
            self.apply_transform()
            #self.augment()
            #unique, counts = np.unique(self.labels, return_counts=True)
            #print(counts)
            # Calculate the unique values and their counts in column 'A'
            unique_values, value_counts = np.unique(self.labels, return_counts=True)

            # Print each unique value and its count
            for value, count in zip(unique_values, value_counts):
                print(f"Value: {value}, Count: {count}")
        elif mode == 'val':
            #unique, counts = np.unique(self.labels, return_counts=True)
            #print(counts)
            self.apply_transform()
            unique_values, value_counts = np.unique(self.labels, return_counts=True)

            # Print each unique value and its count
            for value, count in zip(unique_values, value_counts):
                print(f"Value: {value}, Count: {count}")
        else:
            self.apply_transform()
            
        
    def augment(self):
        
        transform = transforms.Compose([
        transforms.ColorJitter(), 
        transforms.RandomHorizontalFlip(), 
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation((-45,45)), 
        ])
        length = len(self.data)
        for item in range(length):
            if self.labels[item] in [1,4,7,12,14,15]:
              for i in range(15):
                  temp = transform(self.data[item])
                  self.data.append(temp)
                  self.labels.append(self.labels[item])

    def apply_transform(self):
        for item in range(len(self.data)):
            self.data[item] = self.transform(self.data[item])
    
        
    def __len__(self):
            return len(self.data)
    
    def create_data(self):
        data = []
        labels = []
        for img in tqdm(os.listdir(self.ROOT)):
            path = os.path.join(self.ROOT, img)
            if path not in [self.ROOT + "_classes.csv", self.ROOT + "_annotations.csv"]:

              img_data = cv2.imread(path)
              img_data = cv2.resize(img_data, (224, 224))
              label = self.create_label(img)
              if label > 12:
                if label != None:
                    data.append(np.array(img_data)) 
                    labels.append(label)
        return data, labels
    
    def create_label(self,word_label):
       df = pd.read_csv(self.ROOT + "_classes.csv")

       # Specify the value of the first column to filter on
       filter_value = word_label

       # Filter the DataFrame based on the first column value and retrieve all columns except the last
       filtered_row = df.loc[df['filename'] == filter_value].iloc[:, :-1]
       target_value = 1
       target_column_index = None
       for col in filtered_row.columns:
           if filtered_row[col].iloc[0] == target_value:
               target_column_index = filtered_row.columns.get_loc(col)
               break

       return target_column_index-1
      
    def compute_stats(self, data):
        psum    = torch.tensor([0.0, 0.0, 0.0])
        psum_sq = torch.tensor([0.0, 0.0, 0.0])

        # loop through images
        transform = transforms.Compose([transforms.ToTensor()])
        
        inputs = np.zeros((len(data),3,224,224))
        for i in range (len(data)):
            temp = PIL.Image.fromarray(data[i])
            inputs[i] = transform(temp)
            
        inputs = torch.from_numpy(inputs)
        psum    += inputs.sum(axis = [0, 2, 3])
        psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])
            
        count = self.__len__() * 224 * 224
        # mean and std
        total_mean = psum / count
        total_var  = (psum_sq / count) - (total_mean ** 2)
        total_std  = torch.sqrt(total_var)
        
        # output
     
        return total_mean, total_std
        
    def __getitem__(self, idx):
#         print(idx)
        item = self.data[idx]
        label = self.labels[idx]
        return item,label


In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

    
data_dir = '/content/gdrive/My Drive/SeaLifeClassificationDetection'

# TODO: Define transforms for the training data and testing data
transfrom = transforms.Compose([transforms.ToTensor()])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = CustomImageDataset(data_dir + '/train/', mode = 'train', transform=transfrom)
val_data = CustomImageDataset(data_dir + '/valid/', mode = 'val', transform = transfrom)
test_data = CustomImageDataset(data_dir + '/test/',mode = 'test', transform=transfrom)

                                
loader_train = DataLoader(train_data, batch_size=16, shuffle=True)
loader_val = DataLoader(val_data, batch_size=64, shuffle=True)
loader_test = DataLoader(test_data, batch_size=len(test_data))
      

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

100%|██████████| 946/946 [00:11<00:00, 83.87it/s]


Value: 14, Count: 1
Value: 15, Count: 5
Value: 16, Count: 69
Value: 17, Count: 80
Value: 18, Count: 51
Value: 19, Count: 69
Value: 20, Count: 68
Value: 22, Count: 62
Value: 24, Count: 70


100%|██████████| 272/272 [00:08<00:00, 32.70it/s]


Value: 16, Count: 20
Value: 17, Count: 17
Value: 18, Count: 17
Value: 19, Count: 20
Value: 20, Count: 23
Value: 22, Count: 20
Value: 24, Count: 18


100%|██████████| 138/138 [00:03<00:00, 42.42it/s]


In [72]:
USE_GPU = True
dtype = torch.float32 

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print_every = 100

In [73]:
def check_accuracy_part34(loader, model):
    if loader.dataset.mode == 'val':
        print('Checking accuracy on validation set')
    elif loader.dataset.mode == 'train':
        print('Checking accuracy on Train set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        if loader.dataset.mode in ['val', 'test']:
            for x, y in loader:
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                y = y.to(device=device, dtype=torch.long)
                scores = model(x)
                _, preds = scores.max(1)
                num_correct += (preds == y).sum()
                num_samples += preds.size(0)
            acc = float(num_correct) / num_samples
            print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
            return acc
        elif loader.dataset.mode == 'train':
            for x, y in loader:
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                y = y.to(device=device, dtype=torch.long)
                scores = model(x)
                _, preds = scores.max(1)
                num_correct += (preds == y).sum()
                num_samples += preds.size(0)
            acc = float(num_correct) / num_samples
            print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
            return acc
            

In [74]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    accu_sum = []
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            transform = transforms.Compose([transforms.ToTensor()])
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

        print('Iteration %d, loss = %.4f' % (e, loss.item()))
        check_accuracy_part34(loader_val, model)
       
#         print("average = " + str(avg))
        check_accuracy_part34(loader_train, model)


In [75]:
from torchvision.models import resnet18, ResNet18_Weights


class inception_resnet_v2(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=model = resnet18(weights=ResNet18_Weights)
    def forward(self,x):
        out = self.model(x)
        return out

In [76]:

model = inception_resnet_v2()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9, nesterov= True)

train_part34(model, optimizer, epochs=50)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Iteration 0, loss = 2.0435
Checking accuracy on validation set
Got 87 / 135 correct (64.44)
Checking accuracy on Train set
Got 396 / 475 correct (83.37)
Iteration 1, loss = 0.3750
Checking accuracy on validation set
Got 94 / 135 correct (69.63)
Checking accuracy on Train set
Got 465 / 475 correct (97.89)
Iteration 2, loss = 0.2188
Checking accuracy on validation set
Got 100 / 135 correct (74.07)
Checking accuracy on Train set
Got 472 / 475 correct (99.37)
Iteration 3, loss = 0.0986
Checking accuracy on validation set
Got 99 / 135 correct (73.33)
Checking accuracy on Train set
Got 475 / 475 correct (100.00)
Iteration 4, loss = 0.0323
Checking accuracy on validation set
Got 105 / 135 correct (77.78)
Checking accuracy on Train set
Got 475 / 475 correct (100.00)
Iteration 5, loss = 0.0300
Checking accuracy on validation set
Got 107 / 135 correct (79.26)
Checking accuracy on Train set
Got 475 / 475 correct (100.00)
Iteration 6, loss = 0.0727
Checking accuracy on validation set
Got 104 / 135

In [77]:
best_model = model
check_accuracy_part34(loader_test, best_model)


Checking accuracy on test set
Got 52 / 68 correct (76.47)


0.7647058823529411